In [1]:
!pip install transformers
!pip install peft
!pip install pandas
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,AdamW
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Edentns/DataVortexS-10.7B-dpo-v1.11")

# Define BitsAndBytesConfig for quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable loading model in 4-bit
    bnb_4bit_use_double_quant=True,  # Use double quantization for better accuracy
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype=torch.float16  # Compute dtype to use during quantization
)

# Load quantized model with LoRA adjustments
model_name = "Edentns/DataVortexS-10.7B-dpo-v1.11"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Set torch dtype as float16 for the model
    quantization_config=bnb_config  # Apply BitsAndBytesConfig
)

# Prepare model for LoRA training
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
# model.load_state_dict('/content/drive/MyDrive/DBDBDeep/best_model_epoch_1.pt')

In [5]:
# 모델 파일을 로드합니다.
loaded_model = torch.load('/content/drive/MyDrive/DBDBDeep/best_model_epoch_1.pt')

In [6]:
# 모델의 상태 사전을 현재 모델에 적용합니다.
model.load_state_dict(loaded_model)

<All keys matched successfully>

In [7]:
# 모델을 GPU로 옮기기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(48000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer):

In [8]:
# # 사용자 입력을 받아 챗봇이 응답을 반환하도록 함수 만들기

# def return_answer_by_chatbot(user_text):
#     sent = '<usr>' + user_text + '<sys>'
#     input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)   # 입력 문장을 인코딩
#     input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # 텐서로 변환하고 GPU로 이동합니다.

#     # 모델을 사용하여 챗봇의 응답을 생성합. 최대 길이는 50, 샘플링을 수행하며 상위 2개 토큰을 선택.
#     output = model.generate(input_ids, max_length=150, do_sample=True, top_k=2)
#     sentence = tokenizer.decode(output[0].tolist())  # 토큰을 문장으로 디코딩
#     chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
#     return chatbot_response

In [45]:
def return_answer_by_chatbot(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)   # 입력 문장을 인코딩
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)  # 텐서로 변환하고 GPU로 이동합니다.

    # 모델을 사용하여 챗봇의 응답을 생성합니다. 최대 길이는 3000, 샘플링을 수행하며 상위 2개 토큰을 선택.
    output = model.generate(input_ids, max_length=300, do_sample=True, top_k=2)
    sentence = tokenizer.decode(output[0].tolist())  # 토큰을 문장으로 디코딩합니다.

    # 생성된 문장에서 챗봇의 응답 부분을 추출합니다.
    chatbot_response = sentence.split('<sys>')[1].split('<')[0].strip()  # '<sys>' 이후의 부분을 가져온 후 불필요한 태그 제거
    return chatbot_response

In [43]:
user_text ='도배하는 법 알려줘'

In [44]:
return_answer_by_chatbot(user_text)

'도배 방법 은 1. 기존 벽지를  제거하고  벽면을  평탄하게  만든 후, 2. 벽면에  초배지를  부착하고  마르 게 한 후 3. 실배지를  벽면에  부착합니다 .'

In [41]:
return_answer_by_chatbot('좀 더 자세히 알려줘')

'벽지를  붙 일 때는  벽 의 모서리와  수직을  먼 저 확정해야  합니다 . 모서리에  수직을  정확히 맞 추고, 그 위에  첫 번째  패널 을 부착합니다 . 이 때 모서리에  특별 히 신경 을 써 서 접착제를  꼼꼼하게  발라주고 , 수직을  정확히 맞춘  후에  벽지를  부착해야  합니다 . 첫 번째  패널 을 부착'

In [46]:
return_answer_by_chatbot('도배비용 알려주는데 존댓말로 대답해')

'도배 비용 은 보통  도배를  할  평수에  따라  계산합니다 . 예를  들어  도배평수 10평을  기준으로  하면  도배자재  비용 은 약 150,000원, 시공  비용 은 약 250,000원입니다 . 따라 서 총 도배 비용 은 400,000원이  됩니다 . 이 비용 은 도배평수와  시공  업체 에 따라  달라 질 수 있으니  정확한 비용 은 시공  업체 에 문의하는  것이  좋습니다 . 또한 , 시공  업체 에 문의 할 때에는  도배를  시공 할 공간 의 상태 와 시공  일정, 그리고  사용 할 도배재 등을  미리 알려주는  것이  좋습니다 .'

In [33]:
return_answer_by_chatbot('반말 할 수 있니?')

'반말 사용 은 부적절합니다 .'

In [48]:
return_answer_by_chatbot('곰팡이 제거법을 알려줘')

'벽지에  생긴  곰팡이를  제거하는  방법 은 먼 저, 곰팡이가  있는  부분 을 모래지름으로  정확하게  제거한 후, 해당 부분 을 세정제와  함께  깨끗 이 닦 아내야  합니다 . 이렇게  하면  곰팡이가  있던  부분 을 깔끔하게  정리 할 수 있습니다 .'

In [34]:
return_answer_by_chatbot('벽에 곰팡이가 생겼을 때 대처법 알려줘')

'곰팡이가  발생 한 부분 에 베이킹 소다를 뿌리고  식초와 베이킹 소다를 물과  함께  섞 어 만든 용액을  사용하여  닦 아내야  합니다 . 이후  곰팡이가  발생 한 부분 을 건조시킨  후에  다시  도배를  해야  합니다 .'

In [35]:
return_answer_by_chatbot('예쁜 벽지 추천해줘')

"예쁜  벽지로  추천드리 면, '플로럴  벽 지'와 '그래픽  벽 지'가 있습니다 . 플로럴  벽지는  자연적인  요소를  연출하여  공간 을 아늑하게  만들어 줄 뿐만  아니 라, 그래픽  벽지는  다양한 패턴과 컬러를  활용하여  공간 에 풍 부한 분위기 를 조성합니다 ."

In [51]:
return_answer_by_chatbot('구옥 도배할때 주의사항 알려줘')

'구 집 도배 시 주의사항 은 다음과  같습니다 . 1. 구 집 도배시 몰탈  칠 이 되어있는  경우 몰탈  제거 후 도배해야  합니다 . 2. 구 집 도배 시 기존 벽 지 제거 시 벽 면 손상이  발생 할 수 있습니다 . 3. 구 집 도배 시 벽면의  수분 상태  점검 이 필요하며 , 수분이  많은  경우에 는 수분 제거 작업 을 먼 저 해야  합니다 .'

In [52]:
return_answer_by_chatbot('몰탈이 뭔데?')

'몰탈 은 석고보드 와 같이  사용하는  데 주로  석고보드 의 모서리 부분 에 사용되며  석고보드 와 같이  접착 재로 사용됩니다 .'

In [53]:
return_answer_by_chatbot('그럼 몰탈을 제거하는 법을 알려줘')

'몰탈 을 제거하는  방법 은 몰탈  조각 을 떼 어내고  몰탈 이 묻 은 부분 을 세제로  닦아주 는 것입니다 . 몰탈  조각 을 떼 어내기 는 몰탈 의 두께 에 따라 서 시간이  오랜 시간이  걸릴  수도  있습니다 . 또한 , 몰탈 이 묻 은 부분 은 세제로  닦 아내면 되지만  몰탈 이 두꺼 워 세제로  닦 아내기 가 어렵 면 몰탈  제거용 도구를  사용하여  제거해야  합니다 .'